## 한국어 문서에 대한 BERT 활용

다중 언어 BERT 사전학습 모형의 미세조정 학습

BERT는 영어 이외에도 사전학습 모형을 지원한다.
언어 모델로만 학습한 기본 학습 모형으로 bert-base-multilingual-uncased, bert-base-multilingual-cased가 있다.
현재, bert-base-multilingual-cased가 더 많이 추천되고 있다.

<한국어에 특화된 BERT 모형>
1 . SKTBrain의 KoBERT
2 . ETRI의 KorBERT (사용허가협약서가 필요)

In [10]:
#### 영화 리뷰를 읽기

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

df = pd.read_csv("C:/Users/ahyeo/OneDrive/문서/바탕 화면/Project2024/TextMining_ver02/data/daum_movie_review.csv")
#rating이 6보다 작으면 부정, 그 반대면 긍정
y = [0 if rate < 6 else 1 for rate in df.rating]
#데이터셋 분리
X_train_val, X_test, y_train_val, y_test = train_test_split(df.review.tolist(), y, random_state=0)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, random_state = 0)

print('#Train set size :', len(X_train))
print('#Validation set size :', len(X_val))
print('#Test set size :', len(X_test))

#Train set size : 8282
#Validation set size : 2761
#Test set size : 3682


In [8]:
##미세조정을 위해 앞 장에서 배운 Trainer을 사용한다
import torch
from datasets import load_metric

metric = load_metric("accuracy") #accuracy metric load

# 모델 평가시 사용하는 함수
def compute_metrics(eval_pred):
    logits, labels = eval_pred # 모델 logits, 실제 레이블
    predictions = np.argmax(logits, axis = -1) # 모델의 예측값 계산
    return metric.compute(predictions = predictions, references = labels) # 실제 label과 비교해 모델 평가

#데이터셋 정의 -> kobert 설치로 인한 의존성 충돌 ..
class OurDataset(torch.utils.data.Dataset):
    def __init__(self, inputs, labels) :
        self.inputs = inputs
        self.labels = labels

#idx에 해당하는 데이터 샘플을 가져오는 매서드
    def __getitem__(self, idx):
        # 모델에 입력될 데이터를 tensor 형태로 변환, 해당 샘플의 label을 포함한 딕셔너리 반환
        item = {key:torch.tensor(val[idx]) for key, val in self.inputs.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

ImportError: cannot import name 'CommitInfo' from 'huggingface_hub' (C:\Users\ahyeo\anaconda3\envs\TextMining_ver02\lib\site-packages\huggingface_hub\__init__.py)

In [14]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")
print(tokenizer("안녕하세요, 반값습니다."))
inputs = tokenizer("안녕하세요, 반갑습니다.")
print(inputs)

{'input_ids': [101, 9521, 118741, 35506, 24982, 48549, 117, 9321, 118611, 119081, 48345, 119, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
{'input_ids': [101, 9521, 118741, 35506, 24982, 48549, 117, 9321, 118610, 119081, 48345, 119, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


#### 검증 관련 매개변수 정의

Trainer

TrainingArguments
per_device_eval_batch_size : 하이퍼파라미터를 정의, batch size 조절
evaluation_strategy : steps -> eval_steps = 500, 검증을 매 500 스텝마다 실시한다.
step은 각 배치에 대한 학습을 의미하며 epoch 단위로 하고 싶다면, evaluation_strategy 값을 epochs로 준다.
learning_rate_scheduler : 경사하강법에서 경사가 낮은 쪽으로 얼마나 멀리 갈 것인지 결정하는 하이퍼파라미터
학습률이 크면, 학습속도는 빨라지지만 수렴을 놓칠 수 있다. 반대로 학습률이 낮으면, 미세하게 움직여 수렴은 잘 되지만 오래 걸린다.
-> 그러므로 처음에는 학습률을 크게, 갈수록 학습률을 낮게 설정하는 것이 좋다.
get_linear_schedule_with_warmup : 스케줄러에 대한 기본값 설정
warmup : 학습을 낮은 학습률로 몸을 풀어주고 시작하면 학습 효과가 올라간다. (0 -> 1까지 수행하고 점차 내려가는 방식)
weight_decay : 가중치가 너무 커지지 않게 패널티를 주어 과적합을 방지하는 도구 (값이 작을 수록 패널티를 약하게 주는 것)

In [15]:
from transformers import BertForSequenceClassification
from transformers import Trainer, TrainingArguments

#토큰화
#truncation :  입력 시퀀스의 길이가 모델이 처리할 수 있는 최대 길이를 초과할 경우, 초과하는 부분을 자르고(즉, 잘라내고) 최대 길이에 맞춘다.
train_input = tokenizer(X_train, truncation = True, padding = True, return_tensors='pt')
val_input = tokenizer(X_val, truncation = True, padding = True, return_tensors='pt')
test_input = tokenizer(X_test, truncation = True, padding = True, return_tensors='pt')

In [16]:
#데이터셋 생성
train_dataset = OurDataset(train_input, y_train)
val_dataset = OurDataset(val_input, y_train)
test_dataset = OurDataset(test_input, y_train)

In [17]:
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
#Trainer에 사용할 파라미터 지정
training_args = TrainingArguments(
    output_dir = "./results",
    num_train_epochs=2,
    evaluation_strategy="steps",
    eval_steps = 5,
    per_device_train_batch_size= 8,
    per_device_eval_batch_size=16,
    warmup_steps=200, #warmup steps 수
    weight_decay=0.01
)

In [19]:
##trainer 객체 설정
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset = val_dataset, #검증 데이터셋
    compute_metrics= compute_metrics
)

In [20]:
trainer.train()

C:\Users\ahyeo\AppData\Local\Temp\ipykernel_16904\1094331634.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key:torch.tensor(val[idx]) for key, val in self.inputs.items()}


Step,Training Loss,Validation Loss


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\ahyeo\anaconda3\envs\TextMining_ver02\lib\site-packages\IPython\core\interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\ahyeo\AppData\Local\Temp\ipykernel_16904\49973641.py", line 1, in <module>
    trainer.train()
  File "C:\Users\ahyeo\anaconda3\envs\TextMining_ver02\lib\site-packages\transformers\trainer.py", line 1624, in train
    method :meth:`~transformers.Trainer.create_optimizer_and_scheduler` for custom optimizer/scheduler.
  File "C:\Users\ahyeo\anaconda3\envs\TextMining_ver02\lib\site-packages\transformers\trainer.py", line 1961, in _inner_training_loop
    return
  File "C:\Users\ahyeo\anaconda3\envs\TextMining_ver02\lib\site-packages\transformers\trainer.py", line 2902, in training_step
  File "C:\Users\ahyeo\anaconda3\envs\TextMining_ver02\lib\site-packages\transformers\trainer.py", line 2925, in compute_loss
  File "C:\Users\ahyeo\anaconda3\envs\TextMinin

In [ ]:
# trainer.save_model("my_model")

In [ ]:
##성능 측정
trainer.evaluate(eval_dataset = test_dataset)

#### 분석 결과

**GPU가 없어서 오래 걸리기 때문에 .. 책에 나와있는 출력값을 참고하였다.**

학습 데이터셋과 검증 데이터셋에 대한 손실이 모두 꾸준히 줄어든다. (train loss값 0.5 → 0.44까지 줄어듦, val loss도 비슷하게 줄어듦)
학습이 정상적으로 잘 진행되었다고 볼 수 있다.
학습을 조금 더 진행했다면, 더 좋은 성능을 보였을 것 같다. (현재 정확도 : 81.9%)

Trainer는 검증 평가 스텝에서 자동으로 학습 중인 모형을 저장한다.
저장을 할 땐 save_model()를, 불러오고 싶을 땐 from_pretrained() method를 사용하면 된다.

test_dataset으로 성능을 측정하였을 때도 80.1% 정도의 높은 성능을 보여준다.

### KoBERT 사전학습 모형에 대한 파이토치 미세조정

bert 모델이 한국어 문서에 특화되어 있는 모델이 아니고 토크나이저 역시 한국어 구조에 맞게 만들어진 것이 아니기 때문에
한국어에 대해 좋은 성능을 기대하기 어렵다.

KoBERT는 한국어 위키 기반으로 학습해 만든 사전학습 모형으로 다른 모델보다는 더 좋은 성능을 보인다.

In [21]:
#이전 캐시 및 모델 지우기
del model
del trainer
torch.cuda.empty_cache()

In [1]:
#koBERT의 tokenizer 불러오기
#colab을 사용하는 것이 좋을 것 같습니다..
from kobert_tokenizer import KoBERTTokenizer
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')

print(tokenizer.tokenize("안녕하세요. 반갑습니다."))
inputs = tokenizer("안녕하세요. 반갑습니다.")
print(inputs)

C:\Users\ahyeo\anaconda3\envs\TextMining_ver02\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Downloading: 100%|██████████| 371k/371k [00:00<00:00, 1.10MB/s]
Downloading: 100%|██████████| 432/432 [00:00<?, ?B/s] 


['▁안', '녕', '하세요', '.', '▁반', '갑', '습니다', '.']
{'input_ids': [2, 3135, 5724, 7814, 54, 2207, 5345, 6701, 54, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [11]:
#데이터셋 정의
class OurDataset(torch.utils.data.Dataset):
    def __init__(self, inputs, labels) :
        self.inputs = inputs
        self.labels = labels

#idx에 해당하는 데이터 샘플을 가져오는 매서드
    def __getitem__(self, idx):
        # 모델에 입력될 데이터를 tensor 형태로 변환, 해당 샘플의 label을 포함한 딕셔너리 반환
        item = {key:torch.tensor(val[idx]) for key, val in self.inputs.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [12]:
from transformers import BertModel
from torch.utils.data import DataLoader

train_input = tokenizer(X_train, truncation=True, padding = True, return_tensors= 'pt')
val_input = tokenizer(X_val, truncation=True, padding = True, return_tensors= 'pt')
test_input = tokenizer(X_test, truncation=True, padding = True, return_tensors= 'pt')

In [13]:
train_dataset = OurDataset(train_input, y_train)
val_dataset = OurDataset(val_input, y_val)
test_dataset = OurDataset(test_input, y_test)

In [15]:
train_loader = DataLoader(train_dataset, shuffle = True, batch_size = 8)
val_loader = DataLoader(val_dataset, batch_size = 16)
test_loader = DataLoader(test_dataset, batch_size = 16)

In [17]:
#kobert 로드
bert_model = BertModel.from_pretrained('skt/kobert-base-v1')

Downloading: 100%|██████████| 535/535 [00:00<00:00, 268kB/s]
Downloading: 100%|██████████| 369M/369M [01:15<00:00, 4.88MB/s] 


In [18]:
class MyModel(torch.nn.Module):
    def __init__(self, pretrained_model, token_size, num_labels):
        super(MyModel, self).__init__()
        self.token_size = token_size
        self.num_labels = num_labels
        self.pretrained_model = pretrained_model

        #분류기 정의
        self.classifier = torch.nn.Linear(self.token_size, self.num_labels)

    def forward(self, inputs):
        outputs = self.pretrained_model(**inputs)
        # BERT 출력에서 CLS 토큰에 해당하는 부분만 가져온다.
        bert_clf_token = outputs.last_hidden_state[:,0,:]

        return self.classifier(bert_clf_token)

model = MyModel(bert_model, num_labels = 2, token_size=bert_model.config.hidden_size)

In [20]:
from transformers import AdamW, get_linear_schedule_with_warmup
import torch.nn.functional as F
import time

#GPU 설정
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model.to(device)
model.train()

# optimizer 설정
optim = AdamW(model.parameters(), lr = 5e-5, weight_decay=0.01)
criterion = torch.nn.CrossEntropyLoss() #손실함수

num_epochs = 2
total_training_steps = num_epochs
scheduler = get_linear_schedule_with_warmup(optimizer=optim,
                                            num_training_steps=total_training_steps,
                                            num_warmup_steps=200)


start = time.time()
train_loss = 0
eval_steps = 500
step = 0

for epoch in range(num_epochs):
    for batch in train_loader:
        model.train()
        optim.zero_grad() #gradient 초기화

        inputs = {k:v.to(device) for k, v in batch.items() if k!='labels'} #배치에서 label을 제외한 입력만 추출해 GPU로 복사
        labels = batch['labels'].to(device) #batch에서 label 추출
        outputs = model(inputs) #모형 예측

        #손실함수를 통해 두 클래스에 대해 예측하고 각각 비교
        #labels에 one-hot-encoding 적용
        loss = criterion(outputs, F.one_hot(labels, num_classes = 2).float())
        train_loss += loss
        loss.backward()
        optim.step()
        scheduler.step()

        step+=1
        #검증 시간 -> train loss, validation loss 출력
        if step % eval_steps == 0:
            with torch.no_grad():
                val_loss = 0
                model.eval()
                for batch in val_loader:
                    inputs = {k:v.to(device) for k, v in batch.items() if k!='labels'} #배치에서 label을 제외한 입력만 추출해 GPU로 복사
                    labels = batch['labels'].to(device) #batch에서 label 추출
                    outputs = model(inputs) #모형 예측
                    loss = criterion(outputs, F.one_hot(labels, num_classes = 2).float())
                    val_loss += loss
                avg_val_loss = val_loss/len(val_loader)

            avg_train_loss =train_loss/eval_steps
            elapsed = time.time() - start

            print(
                "Step %d, elapsed time: %.2f, train loss : %.4f, validation loss: %.4f"
                % (step, elapsed, avg_train_loss, avg_val_loss)
            )

C:\Users\ahyeo\AppData\Local\Temp\ipykernel_35620\3313682083.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key:torch.tensor(val[idx]) for key, val in self.inputs.items()}


KeyboardInterrupt: 

In [21]:
from datasets import load_metric
metric = load_metric("accuracy")
model.eval()
for batch in test_loader:
    inputs = {k:v.to(device) for k, v in batch.items() if k!='labels'}
    labels = batch['labels'].to(device)

    with torch.no_grad():
        outputs = model(inputs)

    predictions = torch.argmax(outputs, dim = -1)
    metric.add_batch(predictions = predictions, references = labels)

metric.compute()

ImportError: cannot import name 'CommitInfo' from 'huggingface_hub' (C:\Users\ahyeo\anaconda3\envs\TextMining_ver02\lib\site-packages\huggingface_hub\__init__.py)

#### 결과 분석

GPU가 없고 hugging face의 datasets 의존성 문제로 수행 불가

train을 수행하면서 train loss와 validation loss 모두 줄어드는 것을 볼 수 있음
test에 대한 accuracy도 86.90%로 확연히 뛰어난 성능을 보였음